In [6]:
def wraps(func):
    def decorator(inner_func):
        def inner(*args, **kwargs):
            return inner_func(*args, **kwargs)

        inner.__name__ = func.__name__  
        inner.__module__ = func.__module__
        inner.__doc__ = func.__doc__
        return inner            
    return decorator


In [7]:
import functools


def with_arguments(deco):
    
    @wraps(deco)
    def wrapper(*dargs, **dkwargs):
        
        def decorator(func):
            result = deco(func, *dargs, **dkwargs)
            functools.update_wrapper(result, func)
            return result
        
        return decorator
    
    return wrapper

In [10]:
import sys


def trace(func=None, *, handle=sys.stdout): 
    
    if func is None:
        return lambda func: trace(func, handle=handle)
    
    @wraps(func)
    def inner(*args, **kwargs):
        print(func.__doc__, func.__name__, args, kwargs)
        return func(*args, **kwargs)
    
    return inner

@trace(handle=sys.stderr)
def identity(x):
    """I do nothing useful."""
    return x

# identity(42)
identity.__doc__
#print(identity.__doc__)


'I do nothing useful.'

In [19]:
import time


def timethis(func=None, *, n_iter=100):
    if func is None:
        return lambda func: timethis(func, n_iter=n_iter)
    
    @wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, end=" ... ")
        acc = float("inf")
        for i in range(n_iter):
            tick = time.perf_counter()
            result = func(*args, **kwargs)
            acc = min(acc, time.perf_counter() - tick)
        print(acc)
        return result
    return inner

result = timethis(sum)(range(10))

sum ... 1.9999788491986692e-07


In [22]:
def profiled(func):
    @wraps(func)
    def inner(*args, **kwargs):
        inner.ncalls += 1
        return func(*args, **kwargs)
    
    inner.ncalls = 0
    return inner

@profiled
def identity(x):
    return x

identity(1)
identity(2)

identity.ncalls

2

In [33]:
def once(func):
    @wraps(func)
    def inner(*args, **kwargs):
        if not inner.called:
            func(*args, **kwargs)
            inner.called = True
    inner.called = False
    return inner

@once
def initialize_settings():
    print("Settings initialized")


@once
def is_all_good():
    print("All is good")

initialize_settings()
is_all_good()


Settings initialized
All is good


In [34]:
initialize_settings()
is_all_good()

In [38]:
def memoized(func):
    cache = {}

    @wraps(func)
    def inner(*args, **kwargs):
        key = args, tuple(sorted(kwargs.items()))
        if key not in cache:
            cache[key] = func(*args, **kwargs)
        return cache[key]
    return inner

@memoized
def ackermann(m, n):
    if not m:
        return n + 1
    elif not n:
        return ackermann(m - 1, 1)
    else:
        return ackermann(m - 1, ackermann(m, n - 1))


ackermann(3,4)

125

In [45]:
import warnings
def deprecated(func):
    code = func.__code__
    warnings.warn_explicit(
        func.__name__ + " is deprecated.",
        category=DeprecationWarning,
        filename=code.co_filename,
        lineno=code.co_firstlineno + 1
    )
    return func

@deprecated
def pass_func(x):
    return x


In [60]:
import math


def pre(cond, message):
    def wrapper(func):
        @wraps(func)
        def inner(*args, **kwargs):
            assert cond(*args, **kwargs), message
            return func(*args, **kwargs)
        return inner
    return wrapper


def post(cond, message):
    def wrapper(func):
        @wraps(func)
        def inner(*args, **kwargs):
            result = func(*args, **kwargs)
            assert cond(result), message
            return result
        return inner
    return wrapper 

@pre(lambda x: x >= 0, "negative argument")
def checked_log(x):
    return x


@post(lambda x: not math.isnan(x), "not a number")
def something_useful():
    return float("nan")


checked_log(1)
something_useful()


AssertionError: not a number

In [64]:
def square(func):
    return lambda x: func(x * x)


def addsome(func):
    return lambda x: func(x + 42)


@addsome
@square
def identity(x):
    return x

identity(2)

1936